<a href="https://colab.research.google.com/github/PigStep/Restourant-Sentimental-Analys-ML-based/blob/main/experiment_notebook/data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentimental analys. Data preparation

This notebook stands for working with data integration and preparation

In [ ]:
import kagglehub # download data from open sourse

# Download latest version
path = kagglehub.dataset_download("hj5992/restaurantreviews")

print("Path to dataset files:", path)

100%|██████████| 24.1k/24.1k [00:00<00:00, 23.1MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/hj5992/restaurantreviews/versions/1


In [ ]:
import pandas as pd
import numpy as np

dataset = pd.read_csv("/root/.cache/kagglehub/datasets/hj5992/restaurantreviews/versions/1/Restaurant_Reviews.tsv", sep="\t")
dataset.shape

(1000, 2)

In [ ]:
dataset.groupby("Liked").agg({"Liked":"count"})

,Liked
Liked,
0,500
1,500


# Data preparation

In [ ]:
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

def clean_text(text):
    text = text.lower()  # lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # only letters and spaces
    return text

def tokenizeTxt(text):
  tokens = word_tokenize(text)
  return tokens

def remove_stopwords(tokens):
  stop_words = set(stopwords.words('english'))
  negations = {"no", "not", "never", "none", "nobody", "neither", "nor"}
  stop_words = stop_words - negations

  filtered_tokens = [word for word in tokens if word.casefold() not in stop_words]
  return filtered_tokens

def lemmatize(tokens):
  lemmatizer = WordNetLemmatizer()
  lemmas = [lemmatizer.lemmatize(word, pos='v') for word in tokens]  # 'was' → 'be'
  return lemmas

def merge_negative_tokens(tokens):
  skip = False
  negations = {"no", "not", "never", "none", "nobody", "neither", "nor"}
  merge=[]

  for i in range(len(tokens)):
    if skip:
      skip=False
      continue
    if tokens[i] in negations and i+1 <len(tokens):
      skip=True
      merge.append(f"{tokens[i]}_{tokens[i+1]}")
      continue
    merge.append(tokens[i])
  return merge

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
def preprocessText(text):
  text = clean_text(text)
  tokens = tokenizeTxt(text)
  tokens = remove_stopwords(tokens)
  tokens = lemmatize(tokens)
  tokens = merge_negative_tokens(tokens)
  return ' '.join(tokens)

dataset["Review"] = dataset["Review"].apply(preprocessText)
dataset.head()

,Review,Liked
0,wow love place,1
1,crust not_good,0
2,not_tasty texture nasty,0
3,stop late may bank holiday rick steve recommen...,1
4,selection menu great price,1


In [ ]:
dataset.to_csv("dataset.csv")

In [ ]:
%pip install dagshub -q

After data preproccessing lets load data for dataset versioning

In [ ]:
# Upload using the DagsHub client, to a DVC tracked folder also called "data".
# Follow the instructions that appear to authorize the request.

from dagshub import upload_files
upload_files('PigStep/Restourant-Sentimental-Analys-ML-based', 'data')

Output()

Directory upload complete, uploaded 1 files to 
https://dagshub.com/PigStep/Restourant-Sentimental-Analys-ML-based/src/main/data

# Conclusion

This notebook shows data pipeline including:
- lower casing
- nltk tokenization
- special symbols removing
- stopwords removing
- lemmatization
- negative tokens grouping